<a href="https://colab.research.google.com/github/SlayerDraco/personal-assistant-LLM/blob/main/personal_assistant_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
!pip install -q transformers datasets tokenizers accelerate deepspeed

from pathlib import Path

# Create your structure
folders = [
    "config", "tokenizer", "data/raw", "data/processed",
    "training", "model", "inference"
]
for folder in folders:
    Path(folder).mkdir(parents=True, exist_ok=True)

print("✔️ Environment setup complete.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:0

Training the tokeniser


In [2]:
Path("data/raw").mkdir(parents=True, exist_ok=True)

sample_text = """
User: Set a reminder every hour to drink water.
AI: Got it. Hourly hydration threats activated.

User: Take a note that I need to submit the assignment tomorrow.
AI: Noted. Should I also remind you five times before the deadline or let you panic naturally?

User: Schedule a call with Mom at 8 PM.
AI: Scheduled. Tell her I said hi and that you're still alive.

User: What's on my agenda today?
AI: Cry a little, do some work, pretend you're fine. Oh, and a meeting at 3 PM.

User: Draft a message for my professor about the late submission.
AI: "Dear Sir, I deeply regret being a walking deadline disaster..." — want me to send that?

User: Tell me a joke.
AI: You. Trying to wake up before 10 AM.

User: Delete my last note.
AI: Gone. Like your motivation.

User: How's the weather?
AI: Perfect for staying in and questioning your life choices.

User: Play some music.
AI: Playing your favorite: Lo-fi beats to cry and code to.

User: Set an alarm for 7 AM.
AI: Set. But we both know you're hitting snooze.
"""

with open("data/raw/sample_conversations.txt", "w") as f:
    f.write(sample_text.strip())

print("✅ Sample dataset created.")


✅ Sample dataset created.


In [3]:
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path

# Paths
DATA_PATH = "data/raw/"
TOKENIZER_SAVE_PATH = "tokenizer/"
VOCAB_SIZE = 50257  # GPT-style (same as GPT-2)

# Ensure the directory exists - creating if necessary
Path(TOKENIZER_SAVE_PATH).mkdir(parents=True, exist_ok=True)

# Load all .txt files in the raw data folder
def get_training_files(data_path):
    return [str(x) for x in Path(data_path).glob("**/*.txt")]

training_files = get_training_files(DATA_PATH)

# Initialize tokenizer
tokenizer = ByteLevelBPETokenizer()

# Train it
tokenizer.train(
    files=training_files,
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=[
        "<s>", "<pad>", "</s>", "<unk>", "<mask>"
    ]
)

# Save tokenizer
tokenizer.save_model(TOKENIZER_SAVE_PATH)

print(f"✅ Tokenizer trained and saved to `{TOKENIZER_SAVE_PATH}`")

✅ Tokenizer trained and saved to `tokenizer/`


In [4]:
!pip install nltk wordfreq lemminflect

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

from nltk.corpus import wordnet as wn
import lemminflect
from lemminflect import getAllInflections
from wordfreq import top_n_list
import random

all_words = set()

# 1. WordNet base words and inflections
for synset in wn.all_synsets():
    for lemma in synset.lemmas():
        word = lemma.name().lower().replace('_', ' ')
        all_words.add(word)

        # Derivational forms
        if lemma.derivationally_related_forms():
            for related in lemma.derivationally_related_forms():
                all_words.add(related.name().lower().replace('_', ' '))

# 2. Lemminflect inflections
print("Expanding via lemminflect...")
temp_words = list(all_words)
for word in temp_words:
    try:
        inflections = getAllInflections(word)
        for form_list in inflections.values():
            for form in form_list:
                all_words.add(form.lower())
    except:
        continue

# 3. Add high-frequency words from wordfreq (English)
print("Adding top words from wordfreq...")
top_words = top_n_list('en', 10000)
all_words.update(top_words)

print(f"✅ Total unique words generated: {len(all_words)}")

# Optional: Save to a .txt file for tokenizer training
with open("mega_english_words.txt", "w") as f:
    f.write("\n".join(sorted(all_words)))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Expanding via lemminflect...
Adding top words from wordfreq...
✅ Total unique words generated: 176210


In [6]:
from pathlib import Path

# ... other imports and code ...

# Load your raw dataset (e.g., sample_conversations.txt)
with open("data/raw/mew.txt", "r") as file:
    text_data = file.read()

# Encode the entire dataset using the tokenizer
encoded_data = tokenizer.encode(text_data)

# Ensure the 'data/processed' directory exists before saving the file
Path("data/processed").mkdir(parents=True, exist_ok=True)  # This line creates the directory

# You can split your data into smaller chunks if it's too large, or directly save it
# Let's save the encoded data into a file for future use
with open("data/processed/encoded_data.txt", "w") as file:
    file.write("\n".join(map(str, encoded_data.ids)))

print("✅ Data encoded and saved.")


✅ Data encoded and saved.
